In [ ]:
import csv
import json
import logging
import pandas as pd
import requests
import re
from github import Github
from github.GithubException import GithubException

RQ2 What are the developer reactions and sentiments to content containing shared conversations?

In [ ]:
GITHUB_TOKEN = ""
g = Github(GITHUB_TOKEN)

In [ ]:
#human label the data row userful or not, as some PR comments are exact duplicate of the shared conversations.

In [ ]:
df = pd.read_excel("rq2_english_only_useful.xlsx")
len(df)

In [ ]:
df.head(3)

In [ ]:
df["MentionedTextUseful"] = df["MentionedTextUseful"].apply(lambda x: True if x != False else x)
df.head(3)

In [ ]:
df_comments = df[df["MentionedProperty"] == "comments.body"]
len(df_comments)

In [ ]:
df_reviews = df[df["MentionedProperty"] == "reviews.body"]
len(df_reviews)

In [ ]:
df_reviews.head(1)

In [ ]:
df_body = df[df["MentionedProperty"] == "body"]
len(df_body)

In [ ]:
df_body.head()

In [ ]:
def extract_comment_id(url):
    match = re.search(r'#issuecomment-(\d+)', url)
    return match.group(1) if match else None

In [ ]:
#get the comment ids
comment_ids = []
for _, row in df_comments.iterrows():
    comment_id = extract_comment_id(row["comment_url"])
    comment_ids.append(comment_id)

In [ ]:
comment_ids

In [ ]:
df_comment_ids = pd.DataFrame(comment_ids, columns=['comment_ids'])
df_comment_ids

In [ ]:
df_comments["comment_ids"] = comment_ids
df_comments

In [ ]:
all_reactions_comments = []
#react_users = []
#reactions = []
def get_comment_reactions(repo_name, pull_number, comment_id):
    try:
        repo = g.get_repo(f"{repo_name}")
        #print("here")
        pull_request = repo.get_pull(pull_number)
        #print("here")
        comment = pull_request.get_issue_comment(comment_id)
        #print("here")
        reactions = comment.get_reactions()
        reactions_list = list(reactions)
        
        #reaction_dict = {}
        
        if not reactions_list:
            reaction_dict = {}
            reaction_dict["repo_name"] = repo_name
            reaction_dict["pull_number"] = pull_number
            reaction_dict["comment_id"] = comment_id
            reaction_dict["user"] = "None"
            reaction_dict["reaction"] = "None"   
            #react_users.append("None")
            #reactions.append("None")
            all_reactions_comments.append(reaction_dict)
        else:
            for reaction in reactions_list:
                reaction_dict = {}
                reaction_dict["repo_name"] = repo_name
                reaction_dict["pull_number"] = pull_number
                reaction_dict["comment_id"] = comment_id
                reaction_dict["user"] = reaction.user.login
                reaction_dict["reaction"] = reaction.content
                #react_users.append("None")
                #reactions.append("None")
                all_reactions_comments.append(reaction_dict)
                
        #all_reactions_comments.append(reaction_dict)
            
    except GithubException as e:
        if e.status == 404:
            # Handle the 404 error by appending a placeholder
            reaction_dict = {}
            reaction_dict["repo_name"] = repo_name
            reaction_dict["pull_number"] = pull_number
            reaction_dict["comment_id"] = comment_id
            reaction_dict["user"] = "404"
            reaction_dict["reaction"] = "404"
            all_reactions_comments.append(reaction_dict)
            print("error")


In [ ]:
get_comment_reactions("rom-py/rompy", 40, 1742286490)
all_reactions_comments

In [ ]:
#get the reactions
#reactions = []
for _, row in df_comments.iterrows():
    get_comment_reactions(row["RepoName"], int(row["PullNumber"]), int(row["comment_ids"]))

In [ ]:
len(all_reactions_comments)

In [ ]:
all_reactions_comments

In [ ]:
def extract_review_id(url):
    match = re.search(r'#discussion_r(\d+)', url)
    return match.group(1) if match else None

In [ ]:
#get the comment ids
review_ids = []
for _, row in df_reviews.iterrows():
    review_id = extract_review_id(row["comment_url"])
    review_ids.append(review_id)

In [ ]:
review_ids

In [ ]:
df_reviews["comment_ids"] = review_ids
df_reviews

In [ ]:
all_reactions_reviews = []
#react_users = []
#reactions = []
def get_comment_reactions(repo_name, pull_number, comment_id):
    try:
        repo = g.get_repo(f"{repo_name}")
        #print("here")
        pull_request = repo.get_pull(pull_number)
        #print("here")
        comment = pull_request.get_review_comment(comment_id)
        #print("here")
        reactions = comment.get_reactions()
        reactions_list = list(reactions)
        reactions_list
        #reaction_dict = {}
        
        if not reactions_list:
            reaction_dict = {}
            reaction_dict["repo_name"] = repo_name
            reaction_dict["pull_number"] = pull_number
            reaction_dict["comment_id"] = comment_id
            reaction_dict["user"] = "None"
            reaction_dict["reaction"] = "None"   
            #react_users.append("None")
            #reactions.append("None")
            all_reactions_reviews.append(reaction_dict)
        else:
            for reaction in reactions_list:
                reaction_dict = {}
                reaction_dict["repo_name"] = repo_name
                reaction_dict["pull_number"] = pull_number
                reaction_dict["comment_id"] = comment_id
                reaction_dict["user"] = reaction.user.login
                reaction_dict["reaction"] = reaction.content
                #react_users.append("None")
                #reactions.append("None")
                all_reactions_reviews.append(reaction_dict)
                
        #all_reactions_reviews.append(reaction_dict)
            
    except GithubException as e:
        if e.status == 404:
            # Handle the 404 error by appending a placeholder
            reaction_dict = {}
            reaction_dict["repo_name"] = repo_name
            reaction_dict["pull_number"] = pull_number
            reaction_dict["comment_id"] = comment_id
            reaction_dict["user"] = "404"
            reaction_dict["reaction"] = "404"
            all_reactions_reviews.append(reaction_dict)
            print("error")

In [ ]:
get_comment_reactions("mlc-ai/web-stable-diffusion", 52, 1699848356)
all_reactions_reviews

In [ ]:
for _, row in df_reviews.iterrows():
    #print(row["RepoName"])
    #print(int(row["PullNumber"]))
    #print(int(row["comment_ids"]))
    get_comment_reactions(row["RepoName"], int(row["PullNumber"]), int(row["comment_ids"]))

In [ ]:
len(all_reactions_reviews)

In [ ]:
all_reactions_reviews

In [ ]:
all_reactions_body = []
#react_users = []
#reactions = []
def get_body_reactions(repo_name, pull_number):
    try:
        repo = g.get_repo(f"{repo_name}")
        #print("here")
        pull_request = repo.get_pull(pull_number)
        #print("here")
        comments = pull_request.get_issue_comments()
        #print("here")
        
        #reaction_dict = {}
        
        if comments.totalCount == 0:
            reaction_dict = {}
            reaction_dict["repo_name"] = repo_name
            reaction_dict["pull_number"] = pull_number
            reaction_dict["user"] = "None"
            reaction_dict["reaction"] = "None"  
            all_reactions_body.append(reaction_dict)
        else:   
            for comment in comments:
                # Get reactions for each comment
                reactions = comment.get_reactions()
                reactions_list = list(reactions)
                #print(comment)
                #print(reactions_list)

                if not reactions_list:
                    reaction_dict = {}
                    reaction_dict["repo_name"] = repo_name
                    reaction_dict["pull_number"] = pull_number
                    reaction_dict["user"] = "None"
                    reaction_dict["reaction"] = "None"   
                    #react_users.append("None")
                    #reactions.append("None")
                    all_reactions_body.append(reaction_dict)
                else:
                    for reaction in reactions_list:
                        reaction_dict = {}
                        reaction_dict["repo_name"] = repo_name
                        reaction_dict["pull_number"] = pull_number
                        reaction_dict["user"] = reaction.user.login
                        reaction_dict["reaction"] = reaction.content
                        #react_users.append("None")
                        #reactions.append("None")
                        all_reactions_body.append(reaction_dict)

                #all_reactions_body.append(reaction_dict)
            
    except GithubException as e:
        if e.status == 404:
            # Handle the 404 error by appending a placeholder
            reaction_dict = {}
            reaction_dict["repo_name"] = repo_name
            reaction_dict["pull_number"] = pull_number
            reaction_dict["user"] = "404"
            reaction_dict["reaction"] = "404"
            all_reactions_body.append(reaction_dict)
            print("error")

In [ ]:
get_body_reactions("ActivityWatch/aw-webui", 455)
all_reactions_body

In [ ]:
len(df_body)

In [ ]:
for _, row in df_body.iterrows():
    #print(row["RepoName"])
    #print(int(row["PullNumber"]))
    get_body_reactions(row["RepoName"], int(row["PullNumber"]))

In [ ]:
len(all_reactions_body)

In [ ]:
all_reactions_body

In [ ]:
all_reactions_comments

In [ ]:
from collections import Counter
only_reactions_comments = [entry['reaction'] for entry in all_reactions_comments]
reaction_counts = Counter(only_reactions_comments)
print(reaction_counts)

In [ ]:
all_reactions_reviews

In [ ]:
only_reactions_reviews = [entry['reaction'] for entry in all_reactions_reviews]
reaction_counts_reviews = Counter(only_reactions_reviews)
print(reaction_counts_reviews)

In [ ]:
only_reactions_body = [entry['reaction'] for entry in all_reactions_body]
reaction_counts_body = Counter(only_reactions_body)
print(reaction_counts_body)

In [ ]:
only_reactions_comments

In [ ]:
df_comments.head(14)

In [ ]:
df_comments["reactions"] = only_reactions_comments

In [ ]:
df_reviews["reactions"] = only_reactions_reviews

In [ ]:
def assign_reactions(row):
    if row['RepoName'] == 'jefftriplett/frontmatter-cli' and row['PullNumber'] == "6":
        return 'rocket'
    if row['RepoName'] == 'app-sre/qontract-reconcile' and row['PullNumber'] == "3630":
        return '+1'
    if row['RepoName'] == 'livingbio/gpt-fn' and row['PullNumber'] == "4":
        return '404'
    else:
        return 'None'

df_body["reactions"] = df_body.apply(assign_reactions, axis=1)

In [ ]:
df_body

In [ ]:
df_with_reactions = pd.concat([df_comments, df_reviews, df_body], ignore_index=True)
df_with_reactions

In [ ]:
df_with_reactions_sorted = df_with_reactions.sort_values(by='chat_id')

In [ ]:
df_with_reactions_sorted.to_csv("rq2_sorted.csv")

In [ ]:
#Sentiment analysis start from here

In [ ]:
df_useful = df[df["MentionedTextUseful"] == True]
len(df_useful)

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sid = SentimentIntensityAnalyzer()
sentiment_scores = []

for _, row in df_useful.iterrows():
    sentiment_score = sid.polarity_scores(row["MentionedText"])
    sentiment_scores.append(sentiment_score)

#print(sentiment_scores)
sentiment_scores

In [ ]:
sentiments = []
for score in sentiment_scores:
    compound_score = score['compound']
    #print(compound_score)
    if compound_score >= 0.05:
        overall_sentiment = 'Positive'
    elif compound_score <= -0.05:
        overall_sentiment = 'Negative'
    else:
        overall_sentiment = 'Neutral'
        
    if compound_score == 0.0 and score['neu'] == 1.0:
        overall_sentiment = 'Neutral'
        
    sentiments.append(overall_sentiment)
        
sentiments 

In [ ]:
df_sentiments = pd.DataFrame(sentiments)
df_sentiments.value_counts()

In [ ]:
df_useful["sentiment"] = sentiments
df_useful

In [ ]:
df_useful.to_csv("rq2_sentiment_test.csv", encoding = 'utf-8',  index=False)